In [1]:
from datasets import load_dataset

dataset = load_dataset('TheNoob3131/mosquito-data')

Using custom data configuration TheNoob3131--mosquito-data-bf1f4d4d5f427763
Reusing dataset csv (C:\Users\Aswin.Surya24\.cache\huggingface\datasets\TheNoob3131___csv\TheNoob3131--mosquito-data-bf1f4d4d5f427763\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
dataset['train'].features['Photo']
from datasets import Image
dataset = dataset.cast_column('Photo', Image(decode=True))

In [18]:
dataset = dataset.class_encode_column('Classification')
dataset = dataset.rename_column('Classification', 'labels')

ValueError: Column (Classification) not in table columns (['MHM Id', 'labels', 'Container', 'Userid', 'latitude', 'longitude', 'date', 'Photo']).

In [19]:
labels = dataset["train"].features["labels"].names
print(labels)

['aedes', 'culex', 'neither']


In [20]:
id2label = {k:v for k,v in enumerate(labels)}
label2id = {v:k for k,v in enumerate(labels)}
print(id2label)

{0: 'aedes', 1: 'culex', 2: 'neither'}


In [21]:
from transformers import AutoFeatureExtractor, CvtForImageClassification
model_name_or_path = 'microsoft/cvt-13'
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)

loading feature extractor configuration file https://huggingface.co/microsoft/cvt-13/resolve/main/preprocessor_config.json from cache at C:\Users\Aswin.Surya24/.cache\huggingface\transformers\1745fd90e07f290b67705e8a801213fa61b4fab9640f05b898ad57c360cf4314.37be7274d6b5860aee104bb1fbaeb0722fec3850a85bb2557ae9491f17f89433
Feature extractor ConvNextFeatureExtractor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ConvNextFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 224
}



In [22]:
from torchvision.transforms import (
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

transform = Compose(
    [
     RandomResizedCrop(feature_extractor.size),
     RandomHorizontalFlip(),
     ToTensor(),
     normalize
    ]
)

def train_transforms(examples):
  examples["pixel_values"] = [transform(image.convert("RGB")) for image in examples["Photo"]]

  return examples

In [23]:
prepared_ds = dataset.with_transform(train_transforms)

In [36]:
prepared_ds['train'].features['Photo']

Image(decode=True, id=None)

In [42]:
from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

dataloader = DataLoader(prepared_ds["train"], collate_fn=collate_fn, batch_size=4, shuffle=True)

In [43]:
batch = next(iter(dataloader))
for k,v in batch.items():
  if isinstance(v, torch.Tensor):
    print(k, v.shape)

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


In [44]:
prepared_ds['train'].features['Photo']

Image(decode=True, id=None)

In [45]:
from transformers import CvtForImageClassification
labels = dataset['train'].features['labels'].names
#print(len(labels))
model = CvtForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

model.train()

loading configuration file https://huggingface.co/microsoft/cvt-13/resolve/main/config.json from cache at C:\Users\Aswin.Surya24/.cache\huggingface\transformers\4ef5c630651d59551ff5a0071e4bc20b89c24bfb1b2af58442d4b0cb59b79f89.267c6ccda346362800e3a22914da775054354a112accbeff2a01703d0fa98432
Model config CvtConfig {
  "architectures": [
    "CvtForImageClassification"
  ],
  "attention_drop_rate": [
    0.0,
    0.0,
    0.0
  ],
  "cls_token": [
    false,
    false,
    true
  ],
  "depth": [
    1,
    2,
    10
  ],
  "drop_path_rate": [
    0.0,
    0.0,
    0.1
  ],
  "drop_rate": [
    0.0,
    0.0,
    0.0
  ],
  "embed_dim": [
    64,
    192,
    384
  ],
  "id2label": {
    "0": "aedes",
    "1": "culex",
    "2": "neither"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "kernel_qkv": [
    3,
    3,
    3
  ],
  "label2id": {
    "aedes": 0,
    "culex": 1,
    "neither": 2
  },
  "layer_norm_eps": 1e-12,
  "mlp_ratio": [
    4.0,
    4.0,
    4.0
  ],
  "model_type"

CvtForImageClassification(
  (cvt): CvtModel(
    (encoder): CvtEncoder(
      (stages): ModuleList(
        (0): CvtStage(
          (embedding): CvtEmbeddings(
            (convolution_embeddings): CvtConvEmbeddings(
              (projection): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2))
              (normalization): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            )
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (layers): Sequential(
            (0): CvtLayer(
              (attention): CvtAttention(
                (attention): CvtSelfAttention(
                  (convolution_projection_query): CvtSelfAttentionProjection(
                    (convolution_projection): CvtSelfAttentionConvProjection(
                      (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
                      (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [56]:
from datasets import load_metric
import numpy as np


def compute_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
  

    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [57]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./cvt-13-mosquito",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

PyTorch: setting up devices


In [58]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

In [59]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

C:\Users\Aswin.Surya24\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7107
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1780


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,0.912600,0.985329,0.563211,0.588834,0.563162,0.588834
200,0.756900,0.821403,0.594216,0.616458,0.602963,0.616458
300,0.816100,0.838465,0.619309,0.627799,0.541277,0.627799
400,0.834100,0.944966,0.595319,0.560919,0.560559,0.560919
500,0.805400,0.845369,0.623043,0.617621,0.578288,0.617621
600,0.794500,0.822950,0.616606,0.640593,0.621026,0.640593
700,0.799400,0.827959,0.595163,0.614714,0.600080,0.614714
800,0.751300,0.892511,0.616770,0.605118,0.595347,0.605118
900,0.670100,0.866128,0.607960,0.590288,0.598668,0.590288
1000,0.657900,0.790230,0.624717,0.636813,0.618272,0.636813


***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8
Saving model checkpoint to ./cvt-13-mosquito\checkpoint-100
Configuration saved in ./cvt-13-mosquito\checkpoint-100\config.json
Model weights saved in ./cvt-13-mosquito\checkpoint-100\pytorch_model.bin
Feature extractor saved in ./cvt-13-mosquito\checkpoint-100\preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8
Saving model checkpoint to ./cvt-13-mosquito\checkpoint-200
Configuration saved in ./cvt-13-mosquito\checkpoint-200\config.json
Model weights saved in ./cvt-13-mosquito\checkpoint-200\pytorch_model.bin
Feature extractor saved in ./cvt-13-mosquito\checkpoint-200\preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8
Saving model checkpoint to ./cvt-13-mosquito\checkpoint-300
Configuration saved in ./cvt-13-mosquito\checkpoint-300\config.json
Model weights saved in ./cvt-13-mosquito\checkpoint-300\pytorch_model.bin
Feature extrac

***** train metrics *****
  epoch                    =         4.0
  total_flos               = 469000242GF
  train_loss               =      0.7287
  train_runtime            = 13:43:07.18
  train_samples_per_second =       0.576
  train_steps_per_second   =       0.036


In [60]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =       0.64
  eval_f1                 =     0.6209
  eval_loss               =     0.8002
  eval_precision          =     0.6292
  eval_recall             =       0.64
  eval_runtime            = 0:30:25.79
  eval_samples_per_second =      1.884
  eval_steps_per_second   =      0.236
